# Data coverage & quality

In this notebook we will go through the data tables that are used in analytics and modeling and check the coverage and quality of the data.

In [37]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from box import Box
from google.cloud import bigquery

In [38]:
# -------------------- GLOBAL VARIABLES --------------------
PATH_TO_CONFIG_FILE = '../config.yml'

# -------------------- LOAD CONFIGURATION --------------------
# Load the configuration file
config = Box.from_yaml(filename=PATH_TO_CONFIG_FILE)
# Initialize a BigQuery client
client = bigquery.Client(project=config.GCP.PROJECT_ID)
# Set the BigQuery dataset schema
read_schema = config.GCP.READ_SCHEMA
ingestion_schema = config.GCP.INGESTION_SCHEMA
analytics_schema = config.GCP.ANALYTICS_SCHEMA

# Define the color palette
palette = {'good': config.DASHBOARD.COLORS.GOOD_COLOR,
           'warning': config.DASHBOARD.COLORS.WARNING_COLOR,
           'bad': config.DASHBOARD.COLORS.BAD_COLOR}
DEFAULT_VALUE = 'n/a'

In [39]:
def column_coverage(series: pd.Series, default_value: str) -> float:
    """
    Calculate the coverage of a column in percentage.
    :param series: Pandas Series
    :param default_value: Default value to be considered as missing
    :return: Coverage of the column in percentage
    """
    # Calculate the coverage of the column in percentage by dividing the number 
    # of non-default values to the total number of values
    coverage = series[series != default_value].shape[0] / series.shape[0] * 100
    return coverage


def table_health(df: pd.DataFrame, default_value: str) -> pd.DataFrame:
    """
    Calculate the health metrics of a DataFrame.
    :param df: Pandas DataFrame
    :param default_value: Default value to be considered as missing
    :return: Health metrics of the DataFrame
    """
    # list to store the health metrics of the DataFrame
    list_table__health = []
    # Iterate over the columns of the DataFrame
    for column in df.columns:
        # Calculate the coverage of the column
        coverage = column_coverage(df[column], default_value)

        # Append the health metrics to the DataFrame
        list_table__health.append(
            dict(column_name=column,
                 coverage=coverage)
        )

    # Return the health metrics as a DataFrame sorted by coverage
    return pd.DataFrame(list_table__health).sort_values(by='coverage', ascending=False)


# Define the color mapping based on coverage
def get_color_label(coverage):
    """
    Get the color label based on the coverage percentage.
    :param coverage: Coverage percentage
    :return: Color label
    """
    if coverage > 90:
        return 'good'
    elif coverage > 70:
        return 'warning'
    else:
        return 'bad'


def plot_health_metrics(df_source: pd.DataFrame, table_name: str, default_value: str, palette: dict):
    """
    Plot the health metrics of a DataFrame.
    :param df_source: Pandas DataFrame to be analyzed
    :param table_name: Name of the table
    :param default_value: Default value to be considered as missing
    :param palette: Color palette 
    """
    # Calculate the health metrics of the table
    df_health = table_health(df=df_source,
                             default_value=default_value)

    # Visualize the health metrics using a bar chart with the coverage on the y-axis.
    # We use 'good' color for coverage above 90%, 'warning' color for coverage between 70% and 90%
    # and 'bad' color for coverage below 70%.

    # Apply the function to create a new column for color labels
    df_health['color_label'] = df_health['coverage'].apply(get_color_label)

    # Create the bar plot with data labels with hue based on the color labels
    ax = sns.barplot(x='coverage', y='column_name', data=df_health,
                     hue='color_label', palette=palette, dodge=False, legend=False)
    # Add the data labels to the plot (truncated to 0 decimal points)
    for container in ax.containers:
        ax.bar_label(container, fmt='%.0f', label_type='edge')
    plt.xlabel('Coverage (%)')
    plt.ylabel('Column')
    # Set the title of the plot
    plt.title(f'{table_name} - Health Metrics')
    plt.xlim(0, 100)
    plt.show()


## DIM_ARTICLE

Read the `DIM_ARTICLE` table from the BigQuery dataset and output first few rows of the table to understand what the data looks like.

In [ ]:
query_article = f"""
SELECT
    *
FROM
    {read_schema}.DIM_ARTICLE
"""

df_article = client.query(query_article).to_dataframe()
df_article.head(10)

In [ ]:
plot_health_metrics(df_source=df_article,
                    table_name='DIM_ARTICLE',
                    default_value=DEFAULT_VALUE,
                    palette=palette)

### Defining articles for final analysis

After checking base data coverage, we will check how many articles were published by EUTOPIA institutions. This will serve as baseline for further data coverage analysis.
We will then check the following:
1. For how many articles we have the full text available.
2. How many articles are written in English?
3. For how many articles we already have the embeddings available.


In [ ]:
query_articles_eutopia = f"""
SELECT
    COUNT(DISTINCT ARTICLE_SID) AS ARTICLE_COUNT
FROM
    {read_schema}.FCT_COLLABORATION F
WHERE
    IS_EUTOPIAN_PUBLICATION
    AND A.ARTICLE_TITLE <> 'n/a' 
"""

df_articles_eutopia = client.query(query_articles_eutopia).to_dataframe()
n_relevant_articles = df_articles_eutopia.iloc[0, 0]
print(f"Number of articles published by EUTOPIA institutions: {n_relevant_articles}")

In [ ]:
# Check how many articles have full text available
query_articles_full_text = f"""
SELECT
    COUNT(DISTINCT F.ARTICLE_SID) AS ARTICLE_COUNT
FROM
    {read_schema}.FCT_COLLABORATION F
    INNER JOIN {read_schema}.DIM_ARTICLE A 
        ON F.ARTICLE_SID = A.ARTICLE_SID
    INNER JOIN {read_schema}.STG_UNPAYWALL_ARTICLE OA 
        ON A.ARTICLE_DOI = OA.ARTICLE_DOI 
WHERE
    F.IS_EUTOPIAN_PUBLICATION
    AND A.ARTICLE_TITLE <> 'n/a' 
    AND OA.IS_ARTICLE_OPEN_ACCESS
"""

df_articles_full_text = client.query(query_articles_full_text).to_dataframe()
n_articles_full_text = df_articles_full_text.iloc[0, 0]

# Visualize the coverage of articles with full text available
ax = sns.barplot(x=['Open Access', 'Closed'],
                 y=[100 * n_articles_full_text / n_relevant_articles,
                    100 * (n_relevant_articles - n_articles_full_text) / n_relevant_articles],
                 hue=['Open Access', 'Closed'],
                 palette=[palette['good'], palette['bad']])

# Add the data labels to the plot (truncated to 0 decimal points)
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f', label_type='edge')
plt.ylabel('Percentage of relevant articles (%)')
plt.title('Coverage (%) of articles with full text available')
plt.ylim(0, 100)
plt.show()

In [ ]:
# Show article distribution (in %) over languages (df_article.ARTICLE_LANGUAGE) and show 'n/a' with bad color

# Split to 3 groups: English, Non-English and 'n/a'
df_article['ARTICLE_LANGUAGE_GROUP'] = df_article['ARTICLE_LANGUAGE'].apply(
    lambda x: 'English' if x == 'en' else 'Non-English' if x != DEFAULT_VALUE else DEFAULT_VALUE)

# Plot the distribution of articles over languages
ax = sns.barplot(y=df_article['ARTICLE_LANGUAGE_GROUP'].value_counts(normalize=True).index,
                 x=100 * df_article['ARTICLE_LANGUAGE_GROUP'].value_counts(normalize=True).values,
                 hue=df_article['ARTICLE_LANGUAGE_GROUP'].value_counts(normalize=True).index,
                 palette=[palette['good'] if x != DEFAULT_VALUE else palette['bad']
                          for x in df_article['ARTICLE_LANGUAGE_GROUP'].value_counts(normalize=True).index])
plt.xlabel('Language')
plt.ylabel('Percentage of articles (%)')
plt.title('Distribution of articles over languages')
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f', label_type='edge')
plt.xlim(0, 100)
plt.show()

In [ ]:
# Check how for how many articles we have the embeddings available
query_articles_embeddings = f"""
SELECT
    COUNT(DISTINCT F.ARTICLE_SID) AS ARTICLE_COUNT
FROM
    {read_schema}.FCT_COLLABORATION F
    INNER JOIN {read_schema}.DIM_ARTICLE A 
    ON F.ARTICLE_SID = A.ARTICLE_SID 
    INNER JOIN {analytics_schema}.TEXT_EMBEDDING_ARTICLE E
    ON E.DOI = A.ARTICLE_DOI
WHERE
    F.IS_EUTOPIAN_PUBLICATION
    AND A.ARTICLE_TITLE <> 'n/a'
"""

df_articles_embeddings = client.query(query_articles_embeddings).to_dataframe()
n_articles_with_embeddings = df_articles_embeddings.iloc[0, 0]

# Visualize the coverage of articles with embeddings available
ax = sns.barplot(x=['With embeddings', 'Without embeddings'],
                 y=[100 * n_articles_with_embeddings / n_relevant_articles,
                    100 * (n_relevant_articles - n_articles_with_embeddings) / n_relevant_articles],
                 hue=['With embeddings', 'Without embeddings'],
                 palette=[palette['good'], palette['bad']])
# Add the data labels to the plot (truncated to 0 decimal points)
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f', label_type='edge')
plt.ylabel('Percentage of relevant articles (%)')
plt.title('Coverage (%) of articles with embeddings available')
plt.ylim(0, 100)
plt.show()

Given the data coverage, we see several bottlenecks on the data availability. To improve our dataset we will redefine the notion of relevant articles as articles that:
1. Are published by EUTOPIA institutions.
2. Have title available.
3. Have one of the following available: full text, abstract or references.
4. Are written in English.
5. Have the embeddings available. (TODO: this check will be added later as the data is not available yet) 

In [ ]:
query_articles_relevant = f"""
SELECT
    COUNT(DISTINCT F.ARTICLE_SID) AS ARTICLE_COUNT
FROM
    {read_schema}.FCT_COLLABORATION F
    INNER JOIN {read_schema}.DIM_ARTICLE A 
        ON F.ARTICLE_SID = A.ARTICLE_SID
    LEFT JOIN {read_schema}.STG_UNPAYWALL_ARTICLE OA 
        ON A.ARTICLE_DOI = OA.ARTICLE_DOI 
    LEFT JOIN {analytics_schema}.TEXT_EMBEDDING_ARTICLE E
        ON E.DOI = A.ARTICLE_DOI
WHERE
    F.IS_EUTOPIAN_PUBLICATION
    AND A.ARTICLE_TITLE <> 'n/a'
    AND (OA.IS_ARTICLE_OPEN_ACCESS OR A.ARTICLE_ABSTRACT <> 'n/a' OR A.ARTICLE_REFERENCE <> 'n/a')
    AND A.ARTICLE_LANGUAGE = 'en'
    AND E.DOI IS NOT NULL
"""

df_articles_relevant = client.query(query_articles_relevant).to_dataframe()
n_new_relevant_articles = df_articles_relevant.iloc[0, 0]

# Calculate the percentage of relevant articles
pct_relevant_articles = 100
pct_new_relevant_articles = 100 * n_new_relevant_articles / n_relevant_articles

# Visualize the coverage of relevant articles
ax = sns.barplot(y=['All EUTOPIA\narticles', 'Articles with\nsufficient data'],
                 x=[n_relevant_articles, n_new_relevant_articles])

# Add the data labels to the plot including both the value with thousands comma and the percentage of initial definition
for i, value in enumerate([n_relevant_articles, n_new_relevant_articles]):
    pct = 100 if i == 0 else pct_new_relevant_articles
    label = f'{value:,.0f} ({pct:.1f}%)'
    ax.text(value, i, label, ha='center', va='center')

plt.ylabel('Percentage of relevant articles (%)')
plt.title('Coverage (%) of relevant articles relative to initial definition')
plt.show()

## DIM_AUTHOR
Read the `DIM_AUTHOR` table from the BigQuery dataset and output first few rows of the table to understand what the data looks like.

In [ ]:
query_author = f"""
SELECT
    *
FROM
    {read_schema}.DIM_AUTHOR
"""

df_author = client.query(query_author).to_dataframe()
df_author.head(10)

In [ ]:
plot_health_metrics(df_source=df_author,
                    table_name='DIM_AUTHOR',
                    default_value=DEFAULT_VALUE,
                    palette=palette)

## FCT_COLLABORATION
Read the `FCT_COLLABORATION` table from the BigQuery dataset and output first few rows of the table to understand what the data looks like.

In [ ]:
query_collaboration = f"""
SELECT
    *
FROM
    {read_schema}.FCT_COLLABORATION
"""

df_collaboration = client.query(query_collaboration).to_dataframe()
df_collaboration.head(10)

In [ ]:
plot_health_metrics(df_source=df_collaboration,
                    table_name='FCT_COLLABORATION',
                    default_value=DEFAULT_VALUE,
                    palette=palette)

## FCT_COLLABORATION_NOVELTY
Read the `FCT_COLLABORATION_NOVELTY` table from the BigQuery dataset and output first few rows of the table to understand what the data looks like.

In [ ]:
query_collaboration_novelty = f"""
SELECT
    *
FROM
    {read_schema}.FCT_COLLABORATION_NOVELTY
"""

df_collaboration_novelty = client.query(query_collaboration_novelty).to_dataframe()
df_collaboration_novelty.head(10)

In [ ]:
plot_health_metrics(df_source=df_collaboration_novelty,
                    table_name='FCT_COLLABORATION_NOVELTY',
                    default_value=DEFAULT_VALUE,
                    palette=palette)

### Collaboration Novelty Index Distribution
We will check data quality of the `FCT_COLLABORATION_NOVELTY` table by plotting the distribution of the `COLLABORATION_NOVELTY_INDEX` column.

In [ ]:
# Cutoff the outliers top 5 percentiles
df_collaboration_novelty_cut = df_collaboration_novelty[
    (df_collaboration_novelty['COLLABORATION_NOVELTY_INDEX'] <
     df_collaboration_novelty['COLLABORATION_NOVELTY_INDEX'].quantile(
         0.95))]

# Define custom bins
custom_bins = np.linspace(df_collaboration_novelty_cut['COLLABORATION_NOVELTY_INDEX'].min(),
                          df_collaboration_novelty_cut['COLLABORATION_NOVELTY_INDEX'].max(), 30)

# Pre-calculate histogram values
counts, bin_edges = np.histogram(df_collaboration_novelty_cut['COLLABORATION_NOVELTY_INDEX'], bins=custom_bins)

# Plot the pre-calculated histogram values
plt.hist(bin_edges[:-1], bins=bin_edges, weights=counts, alpha=0.7, label='Histogram')
plt.xlabel('Collaboration Novelty Index')
plt.ylabel('Frequency')
plt.title('Distribution of Collaboration Novelty Index')
plt.show()

## FCT_ARTICLE_TOPIC
Read the `FCT_COLLABORATION_TYPE` table from the BigQuery dataset and output first few rows of the table to understand what the data looks like.

In [ ]:
query_article_topic = f"""
SELECT
    *
FROM
    {read_schema}.FCT_ARTICLE_TOPIC
"""

df_article_topic = client.query(query_article_topic).to_dataframe()
df_article_topic.head(10)

In [ ]:
plot_health_metrics(df_source=df_article_topic,
                    table_name='FCT_ARTICLE_TOPIC',
                    default_value=DEFAULT_VALUE,
                    palette=palette)